# Custom circuits:  quantization & spectra
The `Circuit` class allows one to define and analyze custom circuits. The `Circuit` and `SymbolicCircuit` classes work hand in hand to find and separate all periodic and extended degrees of freedom, and to eliminate the non-dynamical cyclic and frozen modes. With this, the symbolic expression of the Hamiltonian is generated in terms of an appropriate choice of variables. 

The `Circuit` class finally performs the numerical diagonalization of the circuit Hamiltonian. Hierarchical diagonalization is utilized for better perfomance, if the user provides a specification of the intended hierarchy of subsystems.

## Defining a custom circuit
Any superconducting circuit consists of **capacitances**, **inductances**, and **Josephson junctions**. 

<div class="alert alert-info">
    
Custom circuit definition
    
A custom circuit is specified via its graph consisting of nodes and branches. Start by labeling all circuit nodes with integer numbers n=1,2,3,... Every branch is a circuit element connecting two nodes.
    
For each branch:
   
- specify branch type: `JJ`, `L`, `C` for Josephson junction, inductance, and capacitance
- give the labels of two nodes connected by the branch
- provide circuit-element parameters (EJ and ECJ, EL, and EC, respectively)
</div>


As a concrete and familiar example, consider the circuit of the zero-pi qubit (nodes already labeled):
   
![zeropi](./zeropi-circ.jpg)

   
The graph of any custom circuit is stored in simple YAML format using the syntax illustrated here:

In [1]:
zp_yaml = """# zero-pi circuit
nodes: 4
branches:
- ["JJ", 1, 2, EJ = 10, 20]
- ["JJ", 3, 4, 10, 20]
- ["L", 2, 3, 0.008]
- ["L", 4, 1, 0.008]
- ["C", 1, 3, 0.02]
- ["C", 2, 4, 0.02]
"""

Alternatively, circuit specifications can be stored and loaded as `yaml` files with the same syntax. The graph description of the circuit is needed for creating an instance of the `Circuit` class. 


### More on syntax for entering custom circuits

The example above illustrates most of the syntax rules to be followed. Each branch is represented by 

```"<branch-type>", <node_1>, <node_2>, <param-1> [, <param-2>]```

**Branch types and parameters:**

- `C`:  branch parameter is the charging energy $E_C = \frac{e^2}{2C}$
- `L`:  branch parameter is the inductive energy $E_L = \frac{\Phi_0^2}{(2\pi)^2 L}$
- `JJ`: branch parameters are the Josephson energy $E_J$ and junction charging energy $E_{CJ}$

*Example:* `"C", 1, 3, 0.02` is a capacitance connecting nodes 1 and node 3, with charging energy 0.02 GHz.

**Symbolic vs. numerical branch branch parameters:**

- Branch parameters can be provided as float values, using the energy units set globally (default: GHz)
- A symbol name can be specified along with the value (e.g., `EJ = 10`). Where appropriate, symbolic expressions are given in terms of such provided symbol names.

TODO: clarify: only initialized once. Do note that if it is initialized again, the default value will be set to the very last initialization (This functionality will be changed to an exception being raised in the future).
    
**Ground node:**

A physical ground node (if to be included in the circuit), always has the label 0. It does not count towards the total number of nodes.

## Creating a `Circuit` object
Using the above string defining the zero-pi qubit, we can easily create a `Circuit` object:

In [2]:
import scqubits as scq
zero_pi = scq.Circuit.from_yaml(zp_yaml, from_file=False, ext_basis="discretized")

Here, `ext_basis` can be set to `"discretized"` or `"harmonic"`, and corresponds to the choice of using "spatial" discretization or decomposition in the harmonic oscillator basis for extended degrees of freedom.


This creation of a `Circuit` automatically runs methods for circuit analysis, quantization, and construction of the circuit Hamiltonian matrix. For instance, we can directly access the **symbolic expression of the circuit Hamiltonian**:

In [8]:
zero_pi.sym_hamiltonian()

-EJ*cos(1.0*θ1 - 1.0*θ3) + 0.01*Q2**2 + 40.0*Q3**2 + 0.04*n1**2 + 0.08*n1*ng_1 + 0.04*ng_1**2 + 0.032*θ2**2 + 0.008*θ3**2 - 10.0*cos(-Φ1 + 1.0*θ1 + 1.0*θ3)

All generalized coordinates are denoted by $\theta_i$; the conjugate charges are given by $Q_i$ for extended degrees of freedom, and by $n_i$ for periodic degrees of freedom.

.. note:
   The coordinates chosen here generally differ from the node variables. In their construction, periodic and extended 
   degrees of freedom are identified and separated. Furthermore, variable elimination is implemented for cyclic and 
   frozen degrees of freedom (if applicable).
   

The **symbolic Lagrangian in terms of node variables** can be accessed via 

In [4]:
zero_pi.sym_lagrangian()

EJ*cos(φ1 - φ2) + 0.5*\dot{φ_1}*(6.256*\dot{φ_1} - 0.006*\dot{φ_2} - 6.25*\dot{φ_3}) + 0.5*\dot{φ_2}*(-0.006*\dot{φ_1} + 6.256*\dot{φ_2} - 6.25*\dot{φ_4}) + 0.5*\dot{φ_3}*(-6.25*\dot{φ_1} + 6.256*\dot{φ_3} - 0.006*\dot{φ_4}) + 0.5*\dot{φ_4}*(-6.25*\dot{φ_2} - 0.006*\dot{φ_3} + 6.256*\dot{φ_4}) - 0.004*(φ1 - φ4)**2 - 0.004*(-φ2 + φ3)**2 + 10.0*cos(Φ1 - φ3 + φ4)

The equivalent expression of the **Lagrangian in terms of the transformed variables** used in `sym_hamiltonian` is:

In [9]:
zero_pi.sym_lagrangian(vars_type="new")

EJ*cos(1.0*θ1 - 1.0*θ3) + 6.256*\dot{θ_1}**2 + 25.0*\dot{θ_2}**2 + 0.006*\dot{θ_3}**2 - 0.032*θ2**2 - 0.008*θ3**2 + 10.0*cos(-Φ1 + 1.0*θ1 + 1.0*θ3)

The classification of the different variables is recorded in `var_categories`:

In [10]:
zero_pi.var_categories

{'periodic': [1], 'extended': [2, 3], 'cyclic': [], 'frozen': []}

Here, the type `osc` refers to the extended modes present in the circuit which do not have any cosine potential. 

The transformation matrix which transforms the new variables defined using $\theta_i$ to node variables defined using $\varphi_i$ can be viewed using the attribute `trans_mat`.

In [ ]:
zero_pi.transformation_matrix

Each of the variable index has its corresponding cutoff defined as an attribute. The list of these attributes can be accessed using:

In [ ]:
zero_pi.cutoff_names

At this point, all we need to do is to define the cutoffs and call `zero_pi.eigenvals()` to get the eigen energies of the Hamiltonian. 

In [ ]:
zero_pi.cutoff_n_1 = 5
zero_pi.cutoff_phi_2 = 10
zero_pi.cutoff_phi_3 = 10
zero_pi.eigenvals()

These eigen values, in this case, are not converged and we need to increase the cutoffs to a large value for convergence. But, this can be expensive for a circuit like zero-pi. Thus, let us try to use hierarchical diagonalization. Looking at the expression generated above from the attribute `hamiltonian_symbolic`, we can see that the variable $\theta_2$ corresponds to the zeta oscillator in zero-pi. Thus, we can pair the variables $1$ and $3$ to a single subsystem. This can be done in the following way. First, define an object which identifies different different subsystems by grouping of the variables. In this object note that, every subsystem is enclosed in `[...]`.

In [ ]:
subsystem_indices = [[1,3], [2]]

The above object indicates that we would want to pair the indices $1, 3$ into a single subsystem and the variable index $2$ into a separate subsystem. Do note that this can be done recursively. For example, a zero pi qubit can be paired with another oscillator with variable index $4$ using `[[[1,3], [2]], [4]]`. Each subsystem needs to be enclosed in square brackets even if it contains a single variable index. This object can then be given to a method `generate_default_trunc_dims` which generates the object for defining the default truncated dimensions for each of the sybsystems.

In [ ]:
from scqubits.core.circuit import generate_default_trunc_dims
generate_default_trunc_dims(HD_indices)

This object returned has the default truncated dimensions for each of the subsystem indexed in the same order as in `subsystem_indices`. This default truncated dims can now be changed accordingly and given as a parameter along with `subsystem_indices` to the method `initiate_circuit`.

In [ ]:
zero_pi.initiate_circuit(subsystem_indices=subsystem_indices, subsystem_trunc_dims=[150, 80])

The object `zero_pi` now initiates subsystems accordingly, whose symbolic hamiltonians and interactions can be viewed using the attribute `subsystems_sym` which refers to the symbolic expressions of the Hamiltonian and the interactions with other subsystems. For example, the hamiltonian of the zero'th subsystem can be accessed using

In [ ]:
zero_pi.subsystems_sym[0][0]

while its interaction can be accessed using

In [ ]:
zero_pi.subsystems_sym[0][1]

Each of the subsystems are also created as an instance of the class `Circuit`. Thus, each subsystem can access all the features as described above. To find the eigenvalues of the subsystem for example: 

In [ ]:
zero_pi.subsystems[0].eigenvals()

Since the circuit was re-initiated with `initiate_circuit`, the cutoffs need to be defined again. And this time, since we are using hierarchical diagonalization we can use higher cutoffs.

In [ ]:
zero_pi.cutoff_n_1 = 10
zero_pi.cutoff_phi_2 = 100
zero_pi.cutoff_phi_3 = 80

All the external flux variables and offset charges can also be set by setting the attributes. To look at the variables for the circuit at hand call: 

In [ ]:
zero_pi.external_flux_vars

In [ ]:
zero_pi.offset_charge_vars

In [ ]:
zero_pi.Φ1 = 0.5
zero_pi.ng_1 = 0.6

Now the eigenvalues and the parameter sweeps can be calculated like any other qubit in scqubits.

In [ ]:
eigs = zero_pi.eigenvals()
eigs - eigs[0]

In [ ]:
import numpy as np
zero_pi.plot_evals_vs_paramvals("Φ1", np.linspace(0,1,21))

# Putting it all together

Putting the code together, this is final result.

In [ ]:
import scqubits as scq
from scqubits.core.circuit import generate_default_trunc_dims
import numpy as np

zero_pi = scq.Circuit.from_input_file("zero-pi-user-manual.yaml", phi_basis="harmonic")

HD_indices = [[1,3], [2]]
generate_default_trunc_dims(HD_indices)

In [ ]:
zero_pi.initiate_circuit(subsystem_indices=subsystem_indices, subsystem_trunc_dims=[150, 80])

zero_pi.cutoff_n_1 = 10
zero_pi.cutoff_phi_2 = 90
zero_pi.cutoff_phi_3 = 150

zero_pi.Φ1 = 0.5
zero_pi.ng_1 = 0.6

eigs = zero_pi.eigenvals()
eigs - eigs[0]

In [ ]:
zero_pi.hamiltonian_symbolic

In [ ]:
zero_pi.plot_evals_vs_paramvals("Φ1", np.linspace(0,1,21))

# Extra features implemented in this module

In many cases, it might be possible that the transformation generated by the code is not what the user prefers. In such an instance the user can define a transformation and provide it as an input to the `initiate_circuit` method. The same applies to the external flux variables and to which branches they get associated to. The user can provide the list of branches to which external flux can be associated to (Do note that any limit has not been implemented in the code to allow the user to associate external flux to more than one branch in a superconducting loop; though in this case the user must make sure the external flux assignment is consistent with what is expected from the circuit as the result might not be physical in some cases). In the case of zero-pi for example user can define custom transformation and closure branches as follows:

In [ ]:
import scqubits as scq
from scqubits.core.circuit import generate_default_trunc_dims
import numpy as np

zero_pi = scq.Circuit.from_input_file("zero-pi-user-manual.yaml", phi_basis="harmonic", basis_completion="simple")

In [ ]:
zero_pi.hamiltonian_symbolic.expand()

In [ ]:
zero_pi.branches

In [ ]:
closure_branches = [zero_pi.branches[3]]

trans_mat = np.array([[ -1,  -1,  1,  1],
                       [ 1,  1,  1,  1],
                       [ 1,  -1, -1,  1],
                       [ -1,  1,  -1,  1]])*0.5
subsystem_indices = [[1,2],[3]]
generate_default_trunc_dims(subsystem_indices)

In [ ]:
zero_pi.initiate_circuit(transformation_matrix=trans_mat, subsystem_indices=subsystem_indices, subsystem_trunc_dims=[50, 100], closure_branches=closure_branches)

In [ ]:
zero_pi.hamiltonian_symbolic.expand()

In [ ]:
zero_pi.Φ1 = 0.0
zero_pi.ng_1 = 0.0

zero_pi.cutoff_n_1 = 6
zero_pi.cutoff_phi_2 = 80
zero_pi.cutoff_phi_3 = 100
eigs = zero_pi.eigenvals()
eigs - eigs[0]

# Plotting the potential
Potential of the circuit can also be plotted using the method `plot_potential`. To see how the potential expression looks like, we can call the attribute `potential_symbolic`.

In [ ]:
zero_pi.potential_symbolic

There are three degrees of freedoms in this potential for which ranges need to be specified. The variables like external flux can also be specified, else they are fetched from the Circuit attributes. Do note that only a maximum of two ranges can be specified as we can only plot in 3D. The method returns a line plot when only one range is specified.

In [ ]:
zero_pi.plot_potential(θ1=np.linspace(-6*np.pi, 6*np.pi), θ2=np.linspace(-6*np.pi, 6*np.pi), θ3 = 6.65)

# Tips when dealing with large circuits

- When the circuit size is larger than a 4 nodes, it is better to not use many symbols for capacitances in the input file as the symbolic matrix conversion required for symbolic Hamiltonian can be expensive. This functionality will be improved in the future when the symbolic Hamiltonian will only be computed after all the parameters are substiuted.
- The methods `from_input_string` and `from_input_file` also take `basis_completion` as a parameter which can take two strings: `"simple"` and `"standard"`. These represent the two methods in which the identity matrix is generated to complete the transformation matrix to transform the node variables to new variables $\theta_i$. `"standard"` refers to an Identity, while `"simple"` refers to a matrix generated using itertools where each column has more than one non zero entries. Depending on the scenario, one of the above two options can result in a simpler Hamiltonian, resulting in a significant improvement in diagonalization time.